---
title: NIRPS Off-axis fiber placement
jupyter:
  jupytext:
    text_representation:
      extension: .qmd
      format_name: quarto
      format_version: '1.0'
      jupytext_version: 1.16.3
  kernelspec:
    display_name: Python 3 (ipykernel)
    language: python
    name: python3
    path: /usr/share/jupyter/kernels/python3
---

In [1]:
import numpy as np
from astropy.coordinates import Angle

This cell sets constants required to convert from astrnomical coordinates to
NIRPS guiding camera coordinates. The plate scale and North angle were calculated by
Gaspare Lo Curto during COMM8 by moving the telescope around a star.
We also use a boolean flag (`EAST_FLIPPED`) to indicate that East is at the right of North (instead of left as in direct imaging convention).

- Measured angle of North coordinate: 148.4+-3.4 degrees.
  - Angle is measured in  the TCCD from horizontal X axis to vertical Y axis counterclockwise. 
  - North is offset by < 90 deg from vertical to the left, so 148.4 - 90 = 58.4

In [2]:
GUIDING_PSCALE = 35.6  # +/- 1.5 mas/pix
GUIDING_NORTH_ANGLE = 58.4  # +/- 3.4 deg
EAST_FLIPPED = True

Below are utility functions to convert between RA/Dec and Sep/PA.
I just took these directly from the [orbitize!](https://orbitize.readthedocs.io/en/latest/index.html) package.

In [3]:
def radec2seppa(ra, dec, mod180=False, err=None):
    """
    Convenience function for converting from
    right ascension/declination to separation/
    position angle.

    Args:
        ra (np.array of float): array of RA values, in mas
        dec (np.array of float): array of Dec values, in mas
        mod180 (Bool): if True, output PA values will be given
            in range [180, 540) (useful for plotting short
            arcs with PAs that cross 360 during observations)
            (default: False)
        err (tuple of np.array): error in sep and pa


    Returns:
        tulple of float: (separation [mas], position angle [deg])

    From the orbitize! package. See https://github.com/sblunt/orbitize/blob/main/LICENSE for the license.

    Copyright (c) 2018, Sarah Blunt, Jason Wang, Isabel Angelo, Henry Ngo, 
    Devin Cody, Logan Pearce, Rob De Rosa, and Eric Nielsen.
    All rights reserved.
    """
    sep = np.sqrt((ra**2) + (dec**2))
    pa = np.degrees(np.arctan2(ra, dec)) % 360.
    if err is not None:
        ra_err, dec_err = err
        sep_err = 0.5*(ra_err+dec_err)
        pa_err = np.degrees(sep_err/sep)

    if mod180:
        pa[pa < 180] += 360

    if err is None:
        return sep, pa
    return sep, pa, sep_err, pa_err


def seppa2radec(sep, pa, err=None):
    """
    Convenience function to convert sep/pa to ra/dec

    Args:
        sep (np.array of float): array of separation in mas
        pa (np.array of float): array of position angles in degrees
        err (tuple of np.array): error in sep and pa

    Returns:
        tuple: (ra [mas], dec [mas])
    """
    pa_rad = np.radians(pa)
    ra = sep * np.sin(pa_rad)
    dec = sep * np.cos(pa_rad)
    if err is not None:
        sep_err, pa_err = err
        pa_err_rad = np.radians(pa_err)
        ra_err = ra * np.sqrt(
            (sep_err/sep)**2 + (np.cos(pa_rad)*pa_err_rad/np.sin(pa_rad))**2
        )
        dec_err = dec * np.sqrt(
            (sep_err/sep)**2 + (np.sin(pa_rad)*pa_err_rad/np.cos(pa_rad))**2
        )

    if err is None:
        return ra, dec
    return ra, dec, ra_err, dec_err

## Definition of available targets

To observe a target, simply add it to the dictionary and set the `target` variable to your target's name
in the next section.
You need to provide one of RA/Dec (`dRA`, `dDec`) and Sep/PA (`sep`, `posang`) so that positions
can be computed. Other keys in the dictionary were likely used only for testing purposes.

The NIRPS will start from the photocenter of the system.
For a high-contrast target, this means the star, in practice.
However, for low-contrast targets, such as Luhman 16, the photocenter will be halfway between the two components.
**In that case, you must adjust the relative coordinates to start from the photocenter, diving by 2**


**Note that these positions were defined around january 2023. Most of these positions will change significantly within a few months or years.**

In [4]:
# Calculated relative coordinates in SEP/PA
targets = {
    # Off-axis binary test for COMM8. Provided by Andres Carmona
    "HD37258": {
        "sep": 990,  # mas
        "posang": 30,
        "star_dec": Angle("-06d09m16.33s"),
    },
    # Nowak+ 2020 (GRAVITY)
    "BetaPic": {
        "dRA": 286.866561546032,
        "dDec": 465.0209786583612,
        "sep": 546.3853353866879,
        "posang": 31.670000285892346,
        "star_dec": Angle("-51d03m59.44s"),
    },
    # Go directly from b to calib pos (double displacement, don't stop at star)
    "BetaPic_b_to_calib": {
        "dRA": -286.866561546032*2,
        "dDec": -465.0209786583612*2,
        "sep": 546.3853353866879*2,
        "posang": 31.670000285892346 + 180,
        "dRA": -284.93 * 2,
        "dDec": -462.35 * 2,
        "sep": 543.09 * 2,
        "posang": 31.64 + 180,
        "star_dec": Angle("-51d03m59.44s"),
    },
    # Opposite from planet Nowak+ 2020 (GRAVITY)
    "BetaPic_calib": {
        "dRA": -286.866561546032,
        "dDec": -465.0209786583612,
        "sep": 546.3853353866879,
        "posang": 31.670000285892346 + 180,
        "star_dec": Angle("-51d03m59.44s"),
    },
    # Go directly from calib to b pos (double displacement)
    "BetaPic_calib_to_b": {
        "dRA": 286.866561546032 * 2,
        "dDec": 465.0209786583612 * 2,
        "sep": 546.3853353866879 * 2,
        "posang": 31.670000285892346,
        "star_dec": Angle("-51d03m59.44s"),
    },
    # NOTE: Dividing by 2 because almost same brightness so we start from photocenter
    "luhman16": {
        "dRA": 820.83 / 2,
        "dDec": -568.39 / 2,
        "sep": 998.42 / 2,
        "posang": 124.70,
        "star_dec": Angle("-53d19m10.08s"),
    },
}

## Setup observation and calculate position

For an observation, we need to specify the target name and the initial fiber position on the
guiding camera (in pixels). **You need to check this manually with the guiding camera after acquisition and set it in the cell below.**

The value `MAX_STEP` defines the maximum step size (in arcsec) that can be performed in a single
fiber offset. During COMM8, we had issues where the AO would be lost when doing too large of a displacement.
The script will automatically break down large displacements in smaller steps. During COMM8, we had success
with 500 mas as the maximum step size, but did not repeat enough to find a robust threshold.

For each step, the target offset is first given (i.e. where we want the fibre to end up).
The "guiding offset" and "guiding position" refer to where the blue guiding circle should be placed.
**This is the opposite of the target fiber position**: the guiding will then move to the star, hence offsetting
the fibre to the desired position.

**Note: During COMM8, we sometimes noticed that the final offset measured in pixel was lower than we intended initially.**
To overcome this, we then used the "Final target position" as comparison and applied an offset opposite to the difference.
Something similar is implemented in the final section of this notebook. **It should only be used if there is a
residual offset left to apply**

In [23]:
tname = "BetaPic_calib"  # Nowak+2020 (GRAVITY)
target = targets[tname]
# NOTE: SET THIS AFTER EVERY ACQUISITION
start_x, start_y = (377, 210)  # in pixel
cosdec = np.cos(target["star_dec"]).value  # Not used below
MAX_STEP = 500  # mas

Functions to calculate pixel offsets from Sep/PA or RA/Dec.

In [24]:
# Calculate pixel offset from SEP/PA
def offset_from_seppa(sep, pa):
    # TODO: Modulo 360?
    sep_pixel = sep / GUIDING_PSCALE  # in mas -> pixel
    posang_rot = (pa - GUIDING_NORTH_ANGLE) % 360

    # Now we have sep in pixel and posang from Y towards X in pixel array
    offx = sep_pixel * np.sin(np.radians(posang_rot))
    offy = sep_pixel * np.cos(np.radians(posang_rot))
    offx, offy = seppa2radec(sep_pixel, posang_rot)

    return offx, offy

# Calculate pixel offset from RA DEC
# Option 1 would be to convert RADEC to SEP/PA
def offset_from_radec(raoff, decoff):
    rot_rad = np.radians(GUIDING_NORTH_ANGLE)
    offx = (raoff * np.cos(rot_rad) - decoff * np.sin(rot_rad)) / GUIDING_PSCALE
    offy = (raoff * np.sin(rot_rad) + decoff * np.cos(rot_rad)) / GUIDING_PSCALE
    return offx, offy


def print_offset_info(offx, offy, ref_x, ref_y):
    print(f"Target offset in X: {offx}")
    print(f"Target offset in Y: {offy}")
    print(f"Current position (X, Y): ({refx}, {refy})")
    print(f"Required guiding offset (X, Y): ({-offx}, {-offy})")
    print(f"Required guiding position (X, Y): ({ref_x - offx}, {ref_y - offy})")

In [25]:
# Compute offset in pixels from Sep/PA or RA/Dec
if "sep" in target and "posang" in target:
    ra, dec = seppa2radec(target["sep"], target["posang"])
    offx_ini, offy_ini = offset_from_seppa(target["sep"], target["posang"])
elif "dRA" in target and "dDec" in target:
    offx_ini, offy_ini = offset_from_radec(target["dRA"], target["dDec"])
else:
    raise KeyError("Expecting one of ('sep', 'posang'), or ('dRA', 'dDec') in target definition")

In [26]:
# Get reference (starting point) in x and y (pixels)
refx, refy = start_x, start_y
offx, offy = offx_ini, offy_ini
finalx, finaly = refx + offx, refy + offy
step_size = np.sqrt(offx**2 + offy**2) * GUIDING_PSCALE  # mas
print(f"Total target offset: {offx, offy}")
print(f"Final target position: {finalx, finaly}")
print("-----------------------------------------------------------")
print("-----------------------------------------------------------")
if step_size > MAX_STEP:
    print("Step size is large. Will have to do incremental steps.")
    print("-----------------------------------------------------------")
    nstep = int((step_size // MAX_STEP) + 1)
    substep_size = step_size / nstep
    for istep in range(nstep):
        print(f"Step {istep+1}")
        offx, offy = offset_from_seppa(substep_size, target["posang"])
        print_offset_info(offx, offy, refx, refy)
        refx -= offx
        refy -= offy
        print("-----------------------------------------------------------")
        print()
else:
    print_offset_info(offx, offy, refx, refy)

Total target offset: (-6.903282509007485, 13.70776448795376)
Final target position: (370.0967174909925, 223.70776448795377)
-----------------------------------------------------------
-----------------------------------------------------------
Step size is large. Will have to do incremental steps.
-----------------------------------------------------------
Step 1
Target offset in X: -3.4516412545037416
Target offset in Y: 6.853882243976878
Current position (X, Y): (377, 210)
Required guiding offset (X, Y): (3.4516412545037416, -6.853882243976878)
Required guiding position (X, Y): (380.4516412545037, 203.14611775602313)
-----------------------------------------------------------

Step 2
Target offset in X: -3.4516412545037416
Target offset in Y: 6.853882243976878
Current position (X, Y): (380.4516412545037, 203.14611775602313)
Required guiding offset (X, Y): (3.4516412545037416, -6.853882243976878)
Required guiding position (X, Y): (383.90328250900745, 196.29223551204626)
--------------

## Additional offset corrections

If there is a residual offset between the current position in pixel and your expected final position.
You can set the current X and Y position below and calculate the additional you need to apply
(for the guiding center), so that fibre is on the final desired position.

In [18]:
# NOTE: ENTER VALUES HERE. Will return an error otherwise
currentx, currenty = None, None
leftx, lefty = finalx - currentx, finaly - currenty
addx, addy = -leftx, -lefty
print(f"Apply additional guiding offset of {-leftx, -lefty}")

TypeError: unsupported operand type(s) for -: 'float' and 'NoneType'